In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
sys.path.append("..")
from all_funcs import util
from model import Generator, Discriminator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
from sklearn.impute import SimpleImputer
tf.keras.backend.set_floatx('float64')
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)

In [2]:
# y_true = [3.]
# y_pred = [0.]

# bce = tf.keras.losses.MeanSquaredError()
# bce(y_true, y_pred).numpy()


In [3]:
from numba import cuda
os.environ['CUDA_VISIBLE_DEVICES']="0"
print(tf.test.is_gpu_available())

True


In [4]:
df=pd.read_csv("../dataset/df_noOutliner_ana.csv",index_col=0)
df, imp_mode, imp_mean=util.FeatureArrange(df)

In [5]:
## slices to data and labels
dataset=df.iloc[:,:-1]#.astype('float32')
org_label=df.iloc[:,-1].to_numpy().reshape(-1,1)#.astype('float32')

In [6]:
sc = MinMaxScaler()
dataset = sc.fit_transform(dataset)
dataset.shape

(4778, 72)

In [7]:
## setting hyperparameter
latent_dim = 72
epochs = 1000
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5
checkpoint_dir = './training_checkpoints'


In [8]:
generator = Generator()
discriminator = Discriminator()

In [9]:
## create Cross Entropy
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [10]:
gen_opt = tf.keras.optimizers.Adam(0.0001,0.5, 0.9)
disc_opt = tf.keras.optimizers.Adam(0.0001,)

In [11]:
@tf.function
def train_discriminator(x, labels):

    noise = tf.random.normal([x.shape[0], latent_dim])

    with tf.GradientTape() as dis_tape:
        gen_data = generator(noise, labels,)
        
        dis_output, labels_fake = discriminator(gen_data, label_out=True)
        
        real_output, labels_real = discriminator(x, label_out=True)
        
        labels_loss = cross_entropy(labels, labels_fake)+\
            cross_entropy(labels, labels_real)
        # formula of Gradient penalty
        x_hat = util.random_weight_average(x, gen_data)
        d_hat = discriminator(x_hat, label_out=False)

        disc_loss = util.discriminator_loss(real_output, dis_output, d_hat, x_hat)+labels_loss
        
    grad_disc = dis_tape.gradient(disc_loss, discriminator.trainable_variables)
    disc_opt.apply_gradients(zip(grad_disc, discriminator.trainable_variables))

    return disc_loss


@tf.function
def train_generator(labels):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape:
        gen_data = generator(noise, labels,)
        dis_output, judge_labels = discriminator(gen_data, label_out=True)
        
        gen_loss = util.generator_loss(dis_output)
        
        # sum all loss
        sum_loss = gen_loss+cross_entropy(labels, judge_labels)

    grad_gen = gen_tape.gradient(sum_loss, generator.trainable_variables)
    gen_opt.apply_gradients(zip(grad_gen, generator.trainable_variables))

    return gen_loss

In [12]:
# save checkpoints
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=gen_opt,
                                 discriminator_optimizer=disc_opt,
                                 generator=generator,
                                 discriminator=discriminator)

In [13]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0
    
    ## create batch dataset 
    training_dataset=tf.data.Dataset.from_tensor_slices((dataset, org_label))\
    .shuffle(buffer_size).batch(batch_size, drop_remainder=True)
    
    for data, labels in training_dataset:
    
        disc_loss += train_discriminator(data, labels)
    
        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss += train_generator(labels)
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1,time.time() - start, 
                                 gen_loss / batch_size, 
                                 disc_loss / (batch_size*n_critic)))


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Time for epoch 1 is 0.07200479507446289 sec - gen_loss = 0.01845508622916501, disc_loss = 0.32911070615305
Time for epoch 2 is 0.012000560760498047 sec - gen_loss = 0.010222103337768263, disc_loss = 0.006148375372876702
Time for epoch 3 is 0.016001224517822266 sec - gen_loss = 0.0015639894969075461, disc_loss = -0.02906001798331062
Time for epoch 4 is 0.012000799179077148 sec - gen_loss = 0.00075276420924551, disc_loss = -0.03704994194810598
Time for epoch 5 is 0.012000560760498047 sec - gen_loss = 0.00

Time for epoch 65 is 0.016000986099243164 sec - gen_loss = 1.4970697862180167e-22, disc_loss = -0.03943686102819259
Time for epoch 66 is 0.012000799179077148 sec - gen_loss = 2.1370036403439416e-29, disc_loss = -0.040991245056167955
Time for epoch 67 is 0.016001224517822266 sec - gen_loss = 1.7126273212624256e-37, disc_loss = -0.04193474635132667
Time for epoch 68 is 0.012000560760498047 sec - gen_loss = 8.899373625291748e-34, disc_loss = -0.042405752658798423
Time for epoch 69 is 0.016001224517822266 sec - gen_loss = 3.160305955830822e-36, disc_loss = -0.04280921358711811
Time for epoch 70 is 0.016000986099243164 sec - gen_loss = 1.7423246663036134e-32, disc_loss = -0.039947147565578336
Time for epoch 71 is 0.012000799179077148 sec - gen_loss = 3.2625789442254166e-37, disc_loss = -0.03918722707313234
Time for epoch 72 is 0.012000560760498047 sec - gen_loss = 8.739923751708763e-38, disc_loss = -0.03569640251733864
Time for epoch 73 is 0.016001224517822266 sec - gen_loss = 1.38080036226

Time for epoch 137 is 0.012000799179077148 sec - gen_loss = 1.9779742614051515e-27, disc_loss = -0.042136372415205986
Time for epoch 138 is 0.012000799179077148 sec - gen_loss = 1.0326280860466459e-29, disc_loss = -0.04097946556420848
Time for epoch 139 is 0.012000799179077148 sec - gen_loss = 1.0444632915603363e-20, disc_loss = -0.04376363071430409
Time for epoch 140 is 0.012000799179077148 sec - gen_loss = 1.2493122803794941e-29, disc_loss = -0.041823822319008365
Time for epoch 141 is 0.012000799179077148 sec - gen_loss = 9.898969665838863e-17, disc_loss = -0.04141234046257479
Time for epoch 142 is 0.016000747680664062 sec - gen_loss = 1.2269085397582384e-32, disc_loss = -0.04266223635619294
Time for epoch 143 is 0.012000799179077148 sec - gen_loss = 8.982189699616034e-20, disc_loss = -0.036772787561282286
Time for epoch 144 is 0.012000560760498047 sec - gen_loss = 2.8836792752094445e-31, disc_loss = -0.047047917215587036
Time for epoch 145 is 0.016001224517822266 sec - gen_loss = 3.

Time for epoch 210 is 0.016000986099243164 sec - gen_loss = 7.7463209884683e-16, disc_loss = -0.042778295429349036
Time for epoch 211 is 0.012000799179077148 sec - gen_loss = 3.746073421656366e-18, disc_loss = -0.04678888584615155
Time for epoch 212 is 0.012000799179077148 sec - gen_loss = 4.510899484229061e-19, disc_loss = -0.043986967728809015
Time for epoch 213 is 0.012000799179077148 sec - gen_loss = 9.339388967303018e-23, disc_loss = -0.04048691104738995
Time for epoch 214 is 0.012000799179077148 sec - gen_loss = 7.919566709082481e-17, disc_loss = -0.04601162153815663
Time for epoch 215 is 0.012000560760498047 sec - gen_loss = 5.5864523984916185e-18, disc_loss = -0.042145149018649734
Time for epoch 216 is 0.012000799179077148 sec - gen_loss = 4.646616791326127e-31, disc_loss = -0.04499360666781578
Time for epoch 217 is 0.012000799179077148 sec - gen_loss = 1.5871186098769293e-17, disc_loss = -0.04012347665504348
Time for epoch 218 is 0.012000799179077148 sec - gen_loss = 3.2945874

Time for epoch 285 is 0.012000560760498047 sec - gen_loss = 1.1390872643633215e-24, disc_loss = -0.042691005127142495
Time for epoch 286 is 0.016001224517822266 sec - gen_loss = 1.5843289985813333e-29, disc_loss = -0.03468711332506418
Time for epoch 287 is 0.012000560760498047 sec - gen_loss = 3.247619892731973e-28, disc_loss = -0.039917393142972706
Time for epoch 288 is 0.012000799179077148 sec - gen_loss = 7.029441225050115e-23, disc_loss = -0.04224413761501732
Time for epoch 289 is 0.012000560760498047 sec - gen_loss = 1.332285849610348e-30, disc_loss = -0.04463162433008851
Time for epoch 290 is 0.012000799179077148 sec - gen_loss = 5.045411213700444e-18, disc_loss = -0.040220538334007246
Time for epoch 291 is 0.012000799179077148 sec - gen_loss = 6.143247761990462e-40, disc_loss = -0.044668568108603375
Time for epoch 292 is 0.012000799179077148 sec - gen_loss = 6.12237903512094e-32, disc_loss = -0.04087347636547169
Time for epoch 293 is 0.02000117301940918 sec - gen_loss = 2.436773

...

Time for epoch 647 is 0.012000560760498047 sec - gen_loss = 7.187377750252232e-35, disc_loss = -0.04621608370965523
Time for epoch 648 is 0.012000799179077148 sec - gen_loss = 3.044296077919871e-36, disc_loss = -0.04451475744679499
Time for epoch 649 is 0.016001224517822266 sec - gen_loss = 2.235832505179689e-29, disc_loss = -0.04393931997444966
Time for epoch 650 is 0.012000322341918945 sec - gen_loss = 1.222537662200984e-22, disc_loss = -0.04884852959242444
Time for epoch 651 is 0.024001598358154297 sec - gen_loss = 3.9022497190465946e-35, disc_loss = -0.04519818717204339
Time for epoch 652 is 0.012000799179077148 sec - gen_loss = 1.839881898064607e-20, disc_loss = -0.04290221390079621
Time for epoch 653 is 0.012000799179077148 sec - gen_loss = 4.2236017539108955e-37, disc_loss = -0.0405172979570501
Time for epoch 654 is 0.012000799179077148 sec - gen_loss = 8.79538781515669e-47, disc_loss = -0.046455442593735005
Time for epoch 655 is 0.012000799179077148 sec - gen_loss = 3.114055032

Time for epoch 718 is 0.012000560760498047 sec - gen_loss = 1.817613804579227e-32, disc_loss = -0.04695721292683107
Time for epoch 719 is 0.016001224517822266 sec - gen_loss = 9.977517953521512e-41, disc_loss = -0.04567067044068946
Time for epoch 720 is 0.012000560760498047 sec - gen_loss = 3.8140307494850784e-42, disc_loss = -0.04597798351874235
Time for epoch 721 is 0.012000799179077148 sec - gen_loss = 5.282318349656924e-37, disc_loss = -0.044725329119954284
Time for epoch 722 is 0.012000799179077148 sec - gen_loss = 5.414140203734854e-39, disc_loss = -0.044063796726463086
Time for epoch 723 is 0.012000799179077148 sec - gen_loss = 1.02575227684096e-33, disc_loss = -0.04640916543343808
Time for epoch 724 is 0.012000799179077148 sec - gen_loss = 8.866686825481359e-39, disc_loss = -0.04096072465844985
Time for epoch 725 is 0.012000799179077148 sec - gen_loss = 6.018575167607365e-25, disc_loss = -0.04845945600316929
Time for epoch 726 is 0.012000799179077148 sec - gen_loss = 4.04262633

Time for epoch 789 is 0.012000799179077148 sec - gen_loss = 5.827499195338491e-19, disc_loss = -0.04568277685331664
Time for epoch 790 is 0.012000799179077148 sec - gen_loss = 4.557200455124565e-28, disc_loss = -0.044943839840508565
Time for epoch 791 is 0.012000799179077148 sec - gen_loss = 3.506710710895181e-30, disc_loss = -0.03997236902223592
Time for epoch 792 is 0.012000560760498047 sec - gen_loss = 6.558215257801979e-26, disc_loss = -0.04538905143346141
Time for epoch 793 is 0.012000799179077148 sec - gen_loss = 3.00728864049854e-29, disc_loss = -0.04642883152363984
Time for epoch 794 is 0.012000799179077148 sec - gen_loss = 5.990788086315433e-32, disc_loss = -0.043073660148780564
Time for epoch 795 is 0.012000799179077148 sec - gen_loss = 1.9248307124917e-40, disc_loss = -0.04293990486900385
Time for epoch 796 is 0.012000799179077148 sec - gen_loss = 1.603015824178315e-33, disc_loss = -0.047420718196402686
Time for epoch 797 is 0.012000560760498047 sec - gen_loss = 8.6978774165

Time for epoch 864 is 0.012000799179077148 sec - gen_loss = 3.2659112090461504e-31, disc_loss = -0.04138149235724677
Time for epoch 865 is 0.012000560760498047 sec - gen_loss = 9.321536133268103e-40, disc_loss = -0.04458765050766973
Time for epoch 866 is 0.012000799179077148 sec - gen_loss = 5.529263807806457e-28, disc_loss = -0.043897199444642675
Time for epoch 867 is 0.012000799179077148 sec - gen_loss = 2.980668443229365e-32, disc_loss = -0.04245456179839738
Time for epoch 868 is 0.012000799179077148 sec - gen_loss = 2.0922966237803647e-32, disc_loss = -0.047031401861636485
Time for epoch 869 is 0.012000560760498047 sec - gen_loss = 7.808225381884906e-31, disc_loss = -0.04588799147712195
Time for epoch 870 is 0.012000799179077148 sec - gen_loss = 8.498693160675323e-40, disc_loss = -0.043904431148672755
Time for epoch 871 is 0.016001224517822266 sec - gen_loss = 1.3251979907497493e-36, disc_loss = -0.0408540416609586
Time for epoch 872 is 0.012000560760498047 sec - gen_loss = 2.63984

Time for epoch 936 is 0.012000799179077148 sec - gen_loss = 1.0178699266353267e-44, disc_loss = -0.04475937607039253
Time for epoch 937 is 0.012000799179077148 sec - gen_loss = 6.9389754770696035e-34, disc_loss = -0.04857000159892122
Time for epoch 938 is 0.012000799179077148 sec - gen_loss = 1.5763381406527395e-31, disc_loss = -0.044738060095813034
Time for epoch 939 is 0.012000799179077148 sec - gen_loss = 2.6220965852346202e-42, disc_loss = -0.04710464810035643
Time for epoch 940 is 0.012000560760498047 sec - gen_loss = 2.68218332381982e-34, disc_loss = -0.04287846945050629
Time for epoch 941 is 0.016001224517822266 sec - gen_loss = 3.6211026089629244e-45, disc_loss = -0.046105413719464534
Time for epoch 942 is 0.012000560760498047 sec - gen_loss = 9.490836210684971e-39, disc_loss = -0.044000016167765395
Time for epoch 943 is 0.012000799179077148 sec - gen_loss = 1.0022255561946492e-39, disc_loss = -0.04657666681464053
Time for epoch 944 is 0.012000799179077148 sec - gen_loss = 1.04

In [14]:
# disc_opt.iterations.numpy()


In [15]:
noise = tf.random.normal([128*2, latent_dim])
zero_label=tf.zeros((128),)
one_label =tf.ones((128),)
all_labels=tf.reshape(tf.stack([zero_label,one_label]),(-1,1))

a=generator(noise,all_labels).numpy()
# (sc.inverse_transform(a))
# for i in sc.inverse_transform(a):
#     print(np.round(i))


If you intended to run this layer in float64, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float32 by default, call `tf.keras.backend.set_floatx('float32')`. To change just this layer, pass dtype='float32' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [16]:
output_dataset=pd.DataFrame(np.round(sc.inverse_transform(a)),columns=[
'BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM', 'PTT1_NM',
'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI', 'age', 'NIHTotal','PPD',
'THD_ID', 'THDA_FL', 'THDH_FL', 'THDI_FL',
'THDAM_FL', 'THDV_FL', 'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL',
'THDOO_FL', 'Gender', 'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
'Hemorrhagic_infarct_ctl', 'cortical_CT', 'subcortical_CT',
'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
'CT_left', 'CT_right', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
'NIHS_10_in', 'NIHS_11_in',  
])
output_dataset

,BT_NM,HR_NM,RR_NM,HB_NM,HCT_NM,PLATELET_NM,WBC_NM,PTT1_NM,PTT2_NM,PTINR_NM,ER_NM,BUN_NM,CRE_NM,BMI,age,NIHTotal,PPD,THD_ID,THDA_FL,THDH_FL,THDI_FL,THDAM_FL,THDV_FL,THDE_FL,THDM_FL,THDR_FL,THDP_FL,THDOO_FL,Gender,cortical_ACA_ctr,cortical_MCA_ctr,subcortical_ACA_ctr,subcortical_MCA_ctr,PCA_cortex_ctr,thalamus_ctr,brainstem_ctr,cerebellum_ctr,Watershed_ctr,Hemorrhagic_infarct_ctr,cortical_ACA_ctl,cortical_MCA_ctl,subcortical_ACA_ctl,subcortical_MCA_ctl,PCA_cortex_ctl,thalamus_ctl,brainstem_ctl,cerebellum_ctl,Watershed_ctl,Hemorrhagic_infarct_ctl,cortical_CT,subcortical_CT,circulation_CT,CT_find,watershed_CT,Hemorrhagic_infarct_CT,CT_left,CT_right,NIHS_1a_in,NIHS_1b_in,NIHS_1c_in,NIHS_2_in,NIHS_3_in,NIHS_4_in,NIHS_5aL_in,NIHS_5bR_in,NIHS_6aL_in,NIHS_6bR_in,NIHS_7_in,NIHS_8_in,NIHS_9_in,NIHS_10_in,NIHS_11_in
0,33.0,51.0,9.0,19.0,19.0,796.0,38.0,28.0,32.0,1.0,677.0,85.0,3.0,41.0,31.0,7.0,113.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0,3.0,3.0,0.0,4.0,3.0,4.0,0.0,2.0,0.0,0.0,0.0
1,34.0,61.0,12.0,18.0,20.0,714.0,32.0,31.0,41.0,1.0,609.0,73.0,4.0,40.0,34.0,11.0,102.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,2.0,3.0,1.0,3.0,3.0,3.0,0.0,1.0,1.0,0.0,0.0
2,34.0,52.0,9.0,19.0,18.0,790.0,34.0,24.0,33.0,1.0,686.0,89.0,3.0,42.0,29.0,7.0,112.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0,3.0,3.0,0.0,4.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0
3,34.0,65.0,11.0,18.0,24.0,747.0,34.0,34.0,46.0,2.0,628.0,83.0,3.0,39.0,37.0,7.0,102.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,2.0,3.0,1.0,4.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0
4,34.0,61.0,13.0,18.0,24.0,741.0,35.0,36.0,57.0,1.0,641.0,81.0,4.0,37.0,33.0,11.0,107.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,3.0,3.0,1.0,3.0,3.0,3.0,0.0,1.0,0.0,0.0,0.0
5,34.0,54.0,11.0,19.0,18.0,743.0,34.0,27.0,40.0,1.0,669.0,81.0,3.0,39.0,36.0,9.0,104.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,3.0,3.0,0.0,3.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0
6,34.0,61.0,12.0,18.0,22.0,739.0,31.0,32.0,49.0,1.0,612.0,82.0,3.0,38.0,37.0,8.0,102.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,0.0,2.0,3.0,1.0,3.0,3.0,3.0,0.0,2.0,0.0,1.0,0.0
7,34.0,53.0,11.0,19.0,21.0,806.0,34.0,37.0,43.0,1.0,624.0,89.0,3.0,42.0,28.0,7.0,109.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,1.0,2.0,0.0,3.0,3.0,1.0,4.0,3.0,4.0,0.0,2.0,0.0,0.0,0.0
8,34.0,61.0,12.0,18.0,22.0,750.0,34.0,31.0,50.0,2.0,638.0,86.0,3.0,40.0,40.0,9.0,107.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,2.0,0.0,2.0,2.0,0.0,3.0,3.0,3.0,0.0,2.0,0.0,0.0,0.0
9,34.0,64.0,12.0,18.0,25.0,713.0,33.0,34.0,53.0,2.0,577.0,81.0,3.0,38.0,38.0,10.0,106.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,3.0,2.0,1.0,1.0,2.0,3.0,1.0,3.0,3.0,3.0,0.0,2.0,0.0,0.0,1.0


In [ ]:
output_dataset.to_csv("../dataset/ouput_dataset/cWGAN_1d_2_models.csv",encoding='utf_8_sig')